In [2]:
# Imports

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor

In [24]:
# Load data

df_train = pd.read_csv("data/train.csv")
df_test = pd.read_csv("data/test.csv")

attributes = ["OverallQual", "YearBuilt", "TotalBsmtSF", "GrLivArea"]
x_train = df_train[attributes]
y_train = df_train["SalePrice"]

In [25]:
rf_reg = RandomForestRegressor()

rf_reg.fit(x_train,y_train)

RandomForestRegressor()

In [26]:
x_test = df_test[attributes].fillna(0)

predictions = rf_reg.predict(x_test)

In [28]:
submission = pd.DataFrame({"Id": df_test["Id"], "SalePrice": np.array(predictions)})
submission.to_csv("submissions/second_test_submission.csv", index=False)